In [4]:
import ipyparallel as ipp
c = ipp.Client()
num_workers = 8

In [10]:
def run_episode(r_goals, th_goals):
        
        #model, env = play_baselines(save_path)
        
        myenv = env.envs[0].env.env # OpenAI uses a bunch of wrappers around my env that don't maintain the member variables I put it
        myenv.num_steps=75
        
        
        dt = myenv._timeStep
        total_time = dt * myenv.num_steps  
    
        ep_state_hist = np.zeros((myenv.num_steps, myenv.observation_space.shape[0]))


        init_states = []
        final_state = []
        
        obs = env.reset()        
        # set the goal for this iteratiom
       
        
        for r_goal in r_goals:
            for th_goal in th_goals:

                x_goal = r_goal*np.cos(th_goal)
                y_goal = r_goal*np.sin(th_goal)

                myenv.goal = [x_goal,y_goal,.1]

                myenv.step(np.zeros(myenv.action_space.shape))

                # run a single episode
                for i in range(myenv.num_steps):
                    actions, _, _, _ = model.step(obs)
                    obs, reward, done, _ = env.step(actions)  
                    ep_state_hist[i,:] = np.copy(obs)
                    if done:
                        break

                # find the last state for all t(states go to zero at episode termination)
                for i, state in enumerate(ep_state_hist):
                    for it, _ in enumerate(ep_state_hist[i:]):

                        t_rem = it*dt

                        init_states.append(torch.cat((torch.tensor(obs.squeeze()), torch.tensor(t_rem,dtype=torch.float32).reshape(1))))


                        if any(ep_state_hist[it,:]):
                            final_states.append(0)          
                        else:
                            final_states.append(1)

        return init_states, final_states


In [7]:
# Generate data for the supervised learning task

num_episodes = 16

r_goals  = [np.random.uniform(low = .1, high = 5, size=int(num_episodes/num_workers)) for _ in range(num_workers)]
th_goals = [np.random.uniform(low =-pi, high = pi, size=int(num_episodes/num_workers)) for _ in range(num_workers)]

#init_states = torch.zeros(t_vals.size, num_episodes, myenv.observation_space.shape[0]+1)  
#final_states = torch.zeros(t_vals.size, num_episodes)

init_states = []
final_states = []


c[:].push(dict(run_episode=run_episode))
c[:].map_sync(lambda i: run_episode(r_goals[i], th_goals[i]), trange(num_workers))
view = c[:]

#c[:].apply(run_episode, env, model)

  0%|          | 0/8 [00:00<?, ?it/s]

CompositeError: one or more exceptions from call to method: <lambda>
[0:apply]: NameError: name 'r_goals' is not defined
[1:apply]: NameError: name 'r_goals' is not defined
[2:apply]: NameError: name 'r_goals' is not defined
[3:apply]: NameError: name 'r_goals' is not defined
.... 4 more exceptions ...